In [31]:
import math as m
import numpy as np

print("Enter number of links:")
l = int(input()) #number of links

dh = np.zeros((l,4)) #empty matrix for DH parameters

for i in range(l-1):
    print("Enter DH parameters for link "+ str(i+1) + " (d, theta, r, alpha):")
    inp = list(map(float, input().strip().split()))[:4]
    dh[i] = inp

H = [0]*(l)
H[0] = np.identity(4)
H0 = np.identity(4) #overall transformation matrix

for i in range(l-1):
   d = dh[i][0]
   theta = dh[i][1]
   r = dh[i][2]
   alpha = dh[i][3]
   Z = [[m.cos(theta), -1*m.sin(theta), 0, 0], #transforation matrix, Z operations
       [m.sin(theta), m.cos(theta), 0, 0],
       [0, 0, 1, d],
       [0, 0, 0, 1]]
   X = [[1, 0, 0, r], #transforation matrix, X operations
       [0, m.cos(alpha), -1*m.sin(alpha), 0],
       [0, m.sin(alpha), m.cos(alpha), 0],
       [0, 0, 0, 1]]
   H[i+1] = np.matmul(Z,X)
   H0 = H0@H[i+1]

Enter number of links:
6
Enter DH parameters for link 1 (d, theta, r, alpha):
0 0.39269908169 1 0
Enter DH parameters for link 2 (d, theta, r, alpha):
0 0.78539816339 1 0
Enter DH parameters for link 3 (d, theta, r, alpha):
0 3.14159265359 0 -1.57079632679
Enter DH parameters for link 4 (d, theta, r, alpha):
0 1.57079632679 0 1.57079632679
Enter DH parameters for link 5 (d, theta, r, alpha):
0 0.52359877559 0 0


In [32]:
ee = [H0[0][3],H0[1][3],H0[2][3]] # The position of the end effector(last origin) wrt base
o = np.zeros(3) #Positions of EE wrt current frame
j = np.zeros((6,l)) #Empty jacobian
joint_type = input("Enter type of joint of each links seperated with space(prismatic/revolute) (include end effector too): ").strip().split()

h = np.identity(4)
for i in range(l):
  h = h@H[i]
  z = [h[0][2], h[1][2], h[2][2]]

  if joint_type[i] == "prismatic":
    j[0][i] = z[0]
    j[1][i] = z[1]
    j[2][i] = z[2]
    j[3][i] = 0
    j[4][i] = 0
    j[5][i] = 0
  else:
    oi = [h[0][3], h[1][3], h[2][3]] #position of current frame origin wrt base origin
    #calculation of o:
    o[0] = ee[0] - oi[0]
    o[1] = ee[1] - oi[1]
    o[2] = ee[2] - oi[2]
    zxoi = (z[1]*o[2]) - (o[1]*z[2])
    zxoj = (o[0]*z[2]) - (z[0]*o[2])
    zxok = (z[0]*o[1]) - (o[0]*z[1])
    zxo = [zxoi, zxoj, zxok] #cross product of z and o
    j[0][i] = zxo[0]
    j[1][i] = zxo[1]
    j[2][i] = zxo[2]
    j[3][i] = z[0]
    j[4][i] = z[1]
    j[5][i] = z[2]

Enter type of joint of each links seperated with space(prismatic/revolute) (include end effector too): rev rev rev rev rev rev


In [33]:
print(j) #Complete manipulator Jacobian

[[-1.30656296e+00 -9.23879533e-01  0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]
 [ 1.30656296e+00  3.82683432e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.23879533e-01
  -3.82683432e-01 -3.82683432e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -3.82683432e-01
  -9.23879533e-01 -9.23879533e-01]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  4.89658886e-12
   4.89658886e-12  4.89658886e-12]]


In [ ]:
print(ee) #End Effector Position

In [ ]:
qdot = list(map(float,input("Enter qdot values: ").strip().split())) # rate of change of joint variables

In [ ]:
xdot = j@qdot
print("Final linear velocities: ", xdot[0], xdot[1], xdot[2],"\n")
print("Final angular velocities: ", xdot[3], xdot[4], xdot[5])